In [1]:
import pandas as pd
train = pd.read_json('data/train.json')
test = pd.read_json('data/test.json')
train_photos = pd.read_csv('data/train_photos.csv').drop(['interest_level'], axis = 1)
test_photos = pd.read_csv('data/test_photos.csv')
train_location = pd.read_csv('data/train_location.csv').drop(['interest_level'], axis = 1)
test_location = pd.read_csv('data/test_location.csv')
train_ids = pd.read_csv('data/train_ids.csv').drop(['interest_level'], axis = 1)
test_ids = pd.read_csv('data/test_ids.csv')
train_created = pd.read_csv('data/train_created.csv').drop(['interest_level'], axis = 1)
test_created = pd.read_csv('data/test_created.csv')
train_basics = pd.read_csv('data/train_basics.csv').drop(['interest_level'], axis = 1)
test_basics = pd.read_csv('data/test_basics.csv')
train_words = pd.read_csv('data/train_words.csv').drop(['interest_level'], axis = 1)
test_words = pd.read_csv('data/test_words.csv')

In [2]:
train_words = train_words[list(train_words)[:4]]
test_words = test_words[list(test_words)[:4]]

In [3]:
train_created_sec = pd.read_csv('data/train_created_sec.csv')
test_created_sec = pd.read_csv('data/test_created_sec.csv')
train_created.created = train_created_sec.created.values
test_created.created = test_created_sec.created.values

In [4]:
train_created.created

0        1466754864000
1        1465733967000
2        1460699683000
3        1461374309000
4        1467220508000
5        1465389700000
6        1460173782000
7        1461720239000
8        1461639259000
9        1466818771000
10       1465961334000
11       1466572559000
12       1466746917000
13       1464932458000
14       1465720293000
15       1464920938000
16       1465619840000
17       1466475127000
18       1465828605000
19       1466750819000
20       1461133968000
21       1467094840000
22       1465729946000
23       1465736715000
24       1465265696000
25       1465908423000
26       1459783311000
27       1466565402000
28       1466143776000
29       1466648675000
             ...      
49322    1466910847000
49323    1467194375000
49324    1464748474000
49325    1466493690000
49326    1465364729000
49327    1465098115000
49328    1467090356000
49329    1461614799000
49330    1465734913000
49331    1465367064000
49332    1466087491000
49333    1464748026000
49334    14

In [5]:
train_k_means = pd.read_csv('ttkaggle/kmean_train_data.csv')[['listing_id', 'cluster']]
test_k_means = pd.read_csv('ttkaggle/kmean_test_data.csv')[['listing_id', 'cluster']]
train_topic_tfidf = pd.read_json('ttkaggle/train_topic_tfidf.json')
test_topic_tfidf = pd.read_json('ttkaggle/test_topic_tfidf.json')
train_topic_tfidf['listing_id'] = train.listing_id.values
test_topic_tfidf['listing_id'] = test.listing_id.values
train_desc_vec = pd.read_csv('ttkaggle/train_desc_vec_mean.csv')
train_feats_vec = pd.read_csv('ttkaggle/train_features_vec_mean.csv')
test_desc_vec = pd.read_csv('ttkaggle/test_desc_vec_mean.csv')
test_feats_vec = pd.read_csv('ttkaggle/test_features_vec_mean.csv')
train_desc_vec.columns = ['desc:' + s for s in list(train_desc_vec)]
train_feats_vec.columns = ['feats:' + s for s in list(train_feats_vec)]
test_desc_vec.columns = ['desc:' + s for s in list(test_desc_vec)]
test_feats_vec.columns = ['feats:' + s for s in list(test_feats_vec)]
train_desc_vec['listing_id'] = train.listing_id.values
test_desc_vec['listing_id'] = test.listing_id.values
train_feats_vec['listing_id'] = train.listing_id.values
test_feats_vec['listing_id'] = test.listing_id.values

In [6]:
train_location = pd.merge(train_location, train_k_means, how = 'left', on= 'listing_id')
test_location = pd.merge(test_location, test_k_means, how = 'left', on= 'listing_id')
train_words_dfs = [train_words, train_topic_tfidf, train_desc_vec, train_feats_vec]
test_words_dfs = [test_words, test_topic_tfidf, test_desc_vec, test_feats_vec]
train_words = reduce(lambda left,right: pd.merge(left,right, how = 'left', on='listing_id'), train_words_dfs)
test_words = reduce(lambda left,right: pd.merge(left,right, how = 'left', on='listing_id'), test_words_dfs)

In [14]:
#combining
train_dfs = [train[['interest_level', 'listing_id']], train_photos, train_location, train_ids, \
           train_created, train_basics, train_words]
test_dfs = [test[['listing_id']], test_photos, test_location, test_ids, \
           test_created, test_basics, test_words]
train_final = reduce(lambda left,right: pd.merge(left,right, how = 'left', on='listing_id'), train_dfs)
test_final = reduce(lambda left,right: pd.merge(left,right, how = 'left', on='listing_id'), test_dfs)
train_final.to_json('train_final.json')
test_final.to_json('test_final.json')

In [8]:
#dummifying
train_dummies = train_final.drop(['interest_level'], axis = 1)
test_dummies = test_final
train_dummies['created'] = pd.to_datetime(train_dummies['created'])
test_dummies['created'] = pd.to_datetime(test_dummies['created'])
change_to_cat = ['created_month_cat', 'created_day_cat', 'created_hour_cat', 'created_weekday_cat', 'bath_cat', 'bed_cat']
for i in change_to_cat:
    train_dummies[i] = train_dummies[i].astype(str)
    test_dummies[i] = test_dummies[i].astype(str)
cat_cols = ['neighborhood_cleaned', 'manager_id_cleaned', 'building_id_cleaned'] + change_to_cat
a = train_dummies[['listing_id'] + cat_cols]
a = pd.get_dummies(a, prefix = ['neigh_dummy_', 'man_dummy_', 'build_dummy_', 'month_dummy_', 'day_dummy_', \
                                'hour_dummy_', 'weekday_dummy_', 'bath_dummy_', 'bed_dummy'])
b = test_dummies[['listing_id'] + cat_cols]
b = pd.get_dummies(b, prefix = ['neigh_dummy_', 'man_dummy_', 'build_dummy_', 'month_dummy_', 'day_dummy_', \
                                'hour_dummy_', 'weekday_dummy_', 'bath_dummy_', 'bed_dummy'])

In [9]:
train_dummies = train_dummies.drop(cat_cols, axis = 1)
train_dummies = pd.merge(train_dummies, a, on = 'listing_id', how = 'left')
test_dummies = test_dummies.drop(cat_cols, axis = 1)
test_dummies = pd.merge(test_dummies, b, on = 'listing_id', how = 'left')
train_dummies['interest_level'] = train_final.interest_level.values

In [50]:
print test_dummies.shape, train_dummies.shape
for i in list(test_dummies):
    x = sum(test_dummies[i].isnull())
    if x > 0:
        print i + ': ' + str(x)
for i in list(train_dummies):
    x = sum(train_dummies[i].isnull())
    if x > 0:
        print i + ': ' + str(x)

(74659, 1487) (49352, 1488)


In [13]:
train_dummies.to_json('train_dummies.json')
test_dummies.to_json('test_dummies.json')

In [11]:
pd.read_csv('train_dummies.csv').created

0        1970-01-01 00:24:26.754864
1        1970-01-01 00:24:25.733967
2        1970-01-01 00:24:20.699683
3        1970-01-01 00:24:21.374309
4        1970-01-01 00:24:27.220508
5        1970-01-01 00:24:25.389700
6        1970-01-01 00:24:20.173782
7        1970-01-01 00:24:21.720239
8        1970-01-01 00:24:21.639259
9        1970-01-01 00:24:26.818771
10       1970-01-01 00:24:25.961334
11       1970-01-01 00:24:26.572559
12       1970-01-01 00:24:26.746917
13       1970-01-01 00:24:24.932458
14       1970-01-01 00:24:25.720293
15       1970-01-01 00:24:24.920938
16       1970-01-01 00:24:25.619840
17       1970-01-01 00:24:26.475127
18       1970-01-01 00:24:25.828605
19       1970-01-01 00:24:26.750819
20       1970-01-01 00:24:21.133968
21       1970-01-01 00:24:27.094840
22       1970-01-01 00:24:25.729946
23       1970-01-01 00:24:25.736715
24       1970-01-01 00:24:25.265696
25       1970-01-01 00:24:25.908423
26       1970-01-01 00:24:19.783311
27       1970-01-01 00:24:26

In [1]:
import pandas as pd
test_words = pd.read_csv('data/test_words.csv')

In [2]:
list(test_words)

['listing_id',
 'num_letters_desc',
 'num_words_desc',
 'num_features',
 'ac',
 'actual apt. photos',
 'bike',
 'brownstone',
 'cats allowed',
 'decorative fireplace',
 'dining room',
 'dishwasher',
 'dogs allowed',
 'doorman',
 'duplex',
 'elevator',
 'exclusive',
 'exposed brick',
 'fireplace',
 'furnished',
 'garden',
 'green building',
 'gym',
 'hardwood',
 'hi rise',
 'high ceiling',
 'high ceilings',
 'highrise',
 'kitchen',
 'laundry',
 'light',
 'live/work',
 'loft',
 'lounge',
 'lounge room',
 'lowrise',
 'luxury building',
 'marble bath',
 'microwave',
 'midrise',
 'multi-level',
 'new construction',
 'newly renovated',
 'no fee',
 'other',
 'outdoor',
 'park',
 'parking',
 'patio',
 'pet friendly',
 'pets',
 'playroom',
 'post war',
 'post-war',
 'prewar',
 'reduced fee',
 'renovated',
 'residents lounge',
 'roof',
 'sauna',
 'shares ok',
 'short term allowed',
 'simplex',
 'stainless steel appliances',
 'storage',
 'sublet',
 'train',
 'valet',
 'view',
 'wheelchair access'